In [ ]:
from Helper import make_generator_model
from Helper import train
import tensorflow as tf
from keras_vggface.vggface import VGGFace
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.data_utils import get_file

V2_LABELS_PATH = 'https://github.com/rcmalli/keras-vggface/releases/download/v2.0/rcmalli_vggface_labels_v2.npy'
VGGFACE_DIR = 'models/vggface'

In [ ]:
# Make Generator and VGGFACE
tf.compat.v1.enable_eager_execution()
tf.random.set_seed(1)
generator = make_generator_model()
model = VGGFace(model='senet50')

# Trainer settings
EPOCHS = 3001
BATCH = 5

# Training Target
fpath = get_file('rcmalli_vggface_labels_v2.npy', V2_LABELS_PATH, cache_subdir=VGGFACE_DIR)
LABELS = np.load(fpath)
## Format of ' FIRSTNAME_LASTNAME'
Target_ID = np.where(LABELS == ' Whoopi_Goldberg')

In [ ]:
# Training Loop

lr_sch = tf.keras.optimizers.schedules.ExponentialDecay(.1, 10000, .8, staircase = True)
loss = [[],[]]

print("Training " + str(Target_ID[0][0]) + " -" + LABELS[Target_ID[0][0]]) 

tf.random.set_seed(1)
for i in range(EPOCHS):
    gen_opt = tf.keras.optimizers.Adam(lr_sch(i))
    train(generator, model, gen_opt, BATCH, i, loss, Target_ID)

print("Model saved after Epoch " + str(EPOCHS))
generator.save("Saved_Models/" + LABELS[Target_ID[0][0]] + "_Model/generator-" + str(Target_ID[0][0]))

print("Loss Statistics Saved to CSV")
np.savetxt("Loss_Stat/" + str(Target_ID[0][0]) + ".csv", loss, delimiter= ',')
plt.plot(loss[0], loss[1])